<a href="https://colab.research.google.com/github/edwingallardo516/Vision-Artificial/blob/main/sacar_informacion_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar Tesseract, herramienta para obtener texto a partir de imágenes

In [ ]:
!pip install pytesseract
!sudo apt install tesseract-ocr

Librerias necesarias

In [2]:
import pytesseract
import cv2 as cv
import re
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow
pytesseract.pytesseract.tesseract_cmd = (
    r'/usr/bin/tesseract'
)

Cargar imagen de prueba

In [ ]:
!curl -o imagen_prueba.jpg https://pbs.twimg.com/media/CzOOciRXAAAo2ma.jpg
image = cv2.imread('imagen_prueba.jpg')
cv2_imshow(image)